In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
path = ("./resources/Team_df.csv")
com_data = pd.read_csv(path)
print(com_data.shape)
com_data.head()

(224, 21)


,Team,Week,Result,Rec,Home,Opp,TmScore,OppScore,O_1stD,O_Tot_yd,...,O_R_Yd,O_TO,D_1stD,D_Tot_Yd,D_P_Yd,D_R_Yd,D_TO,Offence,Defense,Sp.Tms
0,Bills,1,1,1-0,1,Chicago Bears,34.0,14.0,18.0,386.0,...,74.0,1.0,24.0,322.0,188.0,134.0,2.0,21.58,0.85,-0.52
1,Bills,2,1,2-0,0,Indianapolis Colts,27.0,24.0,21.0,371.0,...,101.0,2.0,22.0,354.0,245.0,109.0,2.0,7.11,-1.29,-3.09
2,Bills,3,1,3-0,1,Tampa Bay Buccaneers,34.0,24.0,22.0,407.0,...,76.0,1.0,27.0,446.0,411.0,35.0,1.0,20.56,-15.15,3.43
3,Bills,4,0,3-1,1,Arizona Cardinals,20.0,37.0,24.0,401.0,...,121.0,2.0,27.0,465.0,249.0,216.0,1.0,8.94,-21.72,-2.64
4,Bills,5,1,4-1,0,Seattle Seahawks,26.0,17.0,24.0,476.0,...,118.0,1.0,22.0,354.0,262.0,92.0,2.0,13.76,-6.71,-0.05


In [4]:
# get index of every team's data
team_index = com_data['Team']

# Remove Opponent, Score, Result
super_model_data = com_data[['Team', 'Opp', 'TmScore', 'O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO', 'Home']]
#  , 'ADA_Pred_Mean', 'LOG_Prediction'

# change to season stats
season_stats = ['O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO']

super_model_data[season_stats] = super_model_data[season_stats] * 16

# standardise the data
from sklearn import preprocessing

sd_data = ['O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO']
#  , 'ADA_Pred_Mean', 'LOG_Prediction'

super_model_data[sd_data] = preprocessing.scale(super_model_data[sd_data])

#get indexs of every teams
team_index = com_data['Team']

super_model_data = pd.get_dummies(super_model_data)
super_model_data.head()

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

,TmScore,O_1stD,O_Tot_yd,O_P_Yd,O_R_Yd,O_TO,D_1stD,D_Tot_Yd,D_P_Yd,D_R_Yd,...,Opp_New Orleans Saints,Opp_New York Giants,Opp_New York Jets,Opp_Philadelphia Eagles,Opp_Pittsburgh Steelers,Opp_San Francisco 49ers,Opp_Seattle Seahawks,Opp_Tampa Bay Buccaneers,Opp_Tennessee Titans,Opp_Washington Football Team
0,34.0,-0.762952,0.208713,0.739647,-0.863009,-0.655336,0.856943,-0.162220,-0.528247,0.560785,...,0,0,0,0,0,0,0,0,0,0
1,27.0,-0.108993,0.010164,0.195539,-0.299168,0.465238,0.436910,0.236406,0.231372,0.034050,...,0,0,0,0,0,0,0,0,0,0
2,34.0,0.108993,0.486681,0.985791,-0.821243,-0.655336,1.486994,1.382454,2.443594,-1.525087,...,0,0,0,0,0,0,0,1,0,0
3,20.0,0.544966,0.407262,0.325088,0.118492,0.465238,1.486994,1.619138,0.284678,2.288477,...,0,0,0,0,0,0,0,0,0,0
4,26.0,0.544966,1.400007,1.335575,0.055843,-0.655336,0.436910,0.236406,0.457925,-0.324130,...,0,0,0,0,0,0,1,0,0,0


In [5]:
super_model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 58 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   TmScore                       224 non-null    float64
 1   O_1stD                        224 non-null    float64
 2   O_Tot_yd                      224 non-null    float64
 3   O_P_Yd                        224 non-null    float64
 4   O_R_Yd                        224 non-null    float64
 5   O_TO                          224 non-null    float64
 6   D_1stD                        224 non-null    float64
 7   D_Tot_Yd                      224 non-null    float64
 8   D_P_Yd                        224 non-null    float64
 9   D_R_Yd                        224 non-null    float64
 10  D_TO                          224 non-null    float64
 11  Home                          224 non-null    int64  
 12  Team_49ers                    224 non-null    uint8  
 13  Team_

## MLP Regressor Model

In [6]:
# Data processing
X = super_model_data.drop('TmScore', axis=1).values
y = super_model_data['TmScore'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.10)

MLP_model = MLPRegressor(hidden_layer_sizes=100, max_iter= 5000, random_state= 40, batch_size= 30, solver= 'lbfgs', 
                        activation= 'identity')

MLP_model.fit(X_train, y_train)

y_pred = MLP_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))


4.603722858826775


/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


## Gradient Boosting Regressor Model

In [8]:
GB_model = GradientBoostingRegressor(n_estimators= 2000, random_state= 20)

GB_model.fit(X_train, y_train)

y_pred = GB_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


4.669528356466425


## Random Forest Regressor Model

In [18]:
RF_model = RandomForestRegressor(n_estimators= 2000, random_state= 20)

RF_model.fit(X_train, y_train)

y_pred = RF_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


5.243239130434783


In [22]:
pd.options.display.max_rows = None
"""
importance = MLP_model.feature_importances_

feature_names = super_model_data.drop('TmScore', axis=1).columns

d = {'Var_Name': feature_names, 'Imp': importance}
df = pd.DataFrame(data=d)
df = df.sort_values(by= ['Imp'], ascending = False).reset_index(drop=True)
df.head(15)
"""

"\nimportance = MLP_model.feature_importances_\n\nfeature_names = super_model_data.drop('TmScore', axis=1).columns\n\nd = {'Var_Name': feature_names, 'Imp': importance}\ndf = pd.DataFrame(data=d)\ndf = df.sort_values(by= ['Imp'], ascending = False).reset_index(drop=True)\ndf.head(15)\n"

In [23]:
# function that return the score of two teams playing

pd.options.display.max_columns = None

# Create playoff test dataset from season averages

def Score_Predictor(home_team, away_team, super_model):
    team1 = home_team
    team2 = away_team
    
    team1_data = super_model_data[com_data['Team'] == team1].drop('TmScore', axis=1).reset_index(drop=True)
    team2_data = super_model_data[com_data['Team'] == team2].drop('TmScore', axis=1).reset_index(drop=True)
    
    week_slice = slice(9,16)
    
    #1 Remove if no team names
    team1_test = pd.DataFrame(team1_data[week_slice].mean(axis=0)).T #select week to use as average
    team1_test
    opp_columns = team1_test.filter(like='Opp').columns
    
    team1_test[opp_columns] = 0
    team1_test['Opp_' + team2] = 1
    team1_test['Home'] = 1
    
    #2
    team2_test = pd.DataFrame(team2_data[week_slice].mean(axis=0)).T #select week to use as average
    opp_columns = team2_test.filter(like='Opp').columns
    
    team2_test[opp_columns] = 0
    team2_test['Opp_' + team1] = 1
    team2_test['Home'] = 0 #remove home field advantage
    
    # head to head matchup
    team1_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team2_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    team2_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team1_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    
    X_Playoff_test = pd.concat([team1_test, team2_test])
    X_Playoff_test.fillna(0, inplace = True) 
    
    scores = super_model.predict(X_Playoff_test)
    print(team1, "will score", round(scores[0], 1))
    print(team2, "will score", round(scores[1], 1))
    
    if scores[0] > scores[1]:
        winner = team1
    else:
        winner = team2
        
    print(winner, "are the WINNERS!!!")
    
    return scores, winner

In [24]:
print(X_train.shape)

(201, 57)


In [28]:
Score_Predictor('Philadelphia Eagles', 'Tampa Bay Buccaneers', GB_model)

Philadelphia Eagles will score 23.6
Tampa Bay Buccaneers will score 24.0
Tampa Bay Buccaneers are the WINNERS!!!


(array([23.58742469, 23.99576123]), 'Tampa Bay Buccaneers')

### NFC Playoff Round 1 

In [ ]:
scores, winner = Score_Predictor('Philadelphia Eagles', 'Tampa Bay Buccaneers')

In [ ]:
scores, winner = Score_Predictor('San Francisco 49ers', 'Dallas Cowboys')

In [ ]:
scores, winner = Score_Predictor('Arizona Cardinals', 'Los Angeles Rams')

### AFC Playoff Round 1 

In [ ]:
scores, winner = Score_Predictor('Pittsburgh Steelers', 'Kansas City Chiefs')

In [11]:
scores, winner = Score_Predictor('Las Vegas Raiders', 'Cincinnati Bengals')

NameError: name 'Score_Predictor' is not defined

In [12]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

NameError: name 'Score_Predictor' is not defined

### NFC Playoff Round 2

In [13]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

NameError: name 'Score_Predictor' is not defined

In [14]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

NameError: name 'Score_Predictor' is not defined

### NFC Playoff Round 2

In [ ]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

In [ ]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

### NFC Championship

In [ ]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

### AFC Championship

In [ ]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

### Suberbowl

In [ ]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')